In [1]:
import os
import sys
import torch
import torch.optim as optim
from tensorflow.keras.models import load_model
from torch_geometric.loader import DataLoader as PyGDataLoader

# ========== Add src/ to path if needed ==========
sys.path.append(os.path.join(os.getcwd(), 'src'))

# ========== Imports from project ==========
from dataloader import SatelliteDataset, NUM_GATEWAYS
from model2 import Stage2GNN
from train2 import train_model_with_coverage
from utils import plot_metrics, build_gateway_to_cells_mapping


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(
c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-pa

In [ ]:

# ========== Load Dataset ==========
DATA_FOLDER = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Final folder real data"

file_list = sorted([
    os.path.join(DATA_FOLDER, f)
    for f in os.listdir(DATA_FOLDER)
    if f.endswith('.csv')
])[:10]

train_size = int(0.8 * len(file_list))
train_files, val_files = file_list[:train_size], file_list[train_size:]

train_dataset = SatelliteDataset(train_files)
val_dataset = SatelliteDataset(val_files)

train_loader = PyGDataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = PyGDataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"✅ Loaded {len(train_files)} training files and {len(val_files)} validation files.")

# ========== Load Stage 1 Model ==========
STAGE1_MODEL_PATH = 'stage_1_model.h5'
stage1_model = load_model(STAGE1_MODEL_PATH)

# ========== Build Gateway-to-Cell Mapping ==========
GATEWAY_CELL_CSV = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\cells_with_gateways.csv"
gw_to_cells = build_gateway_to_cells_mapping(GATEWAY_CELL_CSV)
# Count unique cell indices across all gateways
total_cells = len(set(cell for cells in gw_to_cells.values() for cell in cells))



print(f"✅ Loaded gateway-to-cell mapping with {len(gw_to_cells)} gateways covering {total_cells} cells.")

# ========== Build GNN Model ==========
input_dim = 3 + NUM_GATEWAYS * 3  # position + visibility + top3 gateways + neighbors
gnn_model = Stage2GNN(
    input_dim=input_dim,
    sat_feature_dim=111,
    neighbor_feature_dim=NUM_GATEWAYS,
    hidden_dim=256,
    output_dim=NUM_GATEWAYS,
    dropout=0.3,
    use_residual=True
)

optimizer_gnn = optim.Adam(gnn_model.parameters(), lr=0.001)

# ========== Train with Coverage-Aware Loss ==========
results = train_model_with_coverage(
    gnn_model=gnn_model,
    stage1_model=stage1_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer_gnn=optimizer_gnn,
    gw_to_cells=gw_to_cells,              # NOTE: updated argument
    total_cells=total_cells,
    num_epochs=20,
    rounds=15,
    lambda_global=0.2,
    lambda_entropy=0.01,
    lambda_coverage=1.0,
    label_smoothing=0.1
)

(train_losses, val_losses,
 train_top1_acc, train_top3_acc, train_top5_acc,
 val_top1_acc, val_top3_acc, val_top5_acc) = results

# ========== Plot ==========
plot_metrics(train_losses, val_losses,
             train_top1_acc, train_top3_acc, train_top5_acc,
             val_top1_acc, val_top3_acc, val_top5_acc)

# ========== Save ==========
torch.save(gnn_model.state_dict(), 'stage2_loop_gnn_model_with_coverage.pth')
print("✅ GNN model saved with coverage-aware training.")


✅ Loaded 8 training files and 2 validation files.
✅ Loaded gateway-to-cell mapping with 54 gateways covering 4569 cells.
[Epoch 02] Train Loss: 3.8893 | Val Loss: 3.9407 | Top-1: 0.072 / 0.205
[Epoch 04] Train Loss: 3.5441 | Val Loss: 3.4962 | Top-1: 0.250 / 0.302
[Epoch 06] Train Loss: 3.1871 | Val Loss: 3.1068 | Top-1: 0.349 / 0.340
[Epoch 08] Train Loss: 2.8207 | Val Loss: 2.7750 | Top-1: 0.396 / 0.352
[Epoch 10] Train Loss: 2.5123 | Val Loss: 2.5626 | Top-1: 0.422 / 0.332
[Epoch 12] Train Loss: 2.2901 | Val Loss: 2.4535 | Top-1: 0.440 / 0.375
[Epoch 14] Train Loss: 2.1520 | Val Loss: 2.4229 | Top-1: 0.457 / 0.378


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from torch_geometric.data import DataLoader
from tensorflow.keras.models import load_model

from dataloader import prepare_input_for_gnn, build_graph_from_file
from utils import top_k_accuracy
from model2 import Stage2GNN  # use model2 for coverage-aware model

# Constants
NUM_GATEWAYS = 54  # Define this according to your setup

# Paths
DATA_FOLDER = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Final folder real data"
STAGE1_MODEL_PATH = 'stage_1_visible_model.h5'
STAGE2_MODEL_PATH = 'stage2_loop_gnn_model_with_coverage.pth'

# Load Stage 1 model
stage1_model = load_model(STAGE1_MODEL_PATH)

# Initialize and load the GNN model
input_dim = 3 + NUM_GATEWAYS * 3
gnn_model = Stage2GNN(
    input_dim=input_dim,
    sat_feature_dim=111,
    neighbor_feature_dim=NUM_GATEWAYS,
    hidden_dim=256,
    output_dim=NUM_GATEWAYS,
    dropout=0.3,
    use_residual=True
)
gnn_model.load_state_dict(torch.load(STAGE2_MODEL_PATH))
gnn_model.eval()

# List all .csv files
file_list = sorted([
    os.path.join(DATA_FOLDER, f)
    for f in os.listdir(DATA_FOLDER)
    if f.endswith('.csv')
])

# Define test ranges
test_ranges = {
    "Train Range 0-5": (0, 5),
    "Train Range 5-10": (5, 10),
    "Test Range 100-105": (100, 105),
    "Test Range 200-205": (200, 205)
}

# Function to evaluate test datasets
def evaluate_test_set(name, start, end):
    print(f"\n===== Evaluating {name} (Files {start} to {end}) =====")
    test_files = file_list[start:end]
    test_graphs = [build_graph_from_file(f) for f in test_files if build_graph_from_file(f) is not None]
    test_loader = DataLoader(test_graphs, batch_size=1, shuffle=False)

    total_top1, total_top3, total_top5, total_samples = 0, 0, 0, 0
    predictions = []

    with torch.no_grad():
        for data in test_loader:
            input_features = data.x[:, :57].cpu().numpy()
            stage1_preds = stage1_model.predict(input_features, verbose=0)
            top3_model1 = np.argsort(stage1_preds, axis=1)[:, -3:]

            binary_preds_model1 = np.zeros_like(stage1_preds)
            for i, idx in enumerate(top3_model1):
                binary_preds_model1[i, idx] = 1

            gnn_input = prepare_input_for_gnn(data, torch.tensor(binary_preds_model1, dtype=torch.float))
            preds = gnn_model(gnn_input.x, gnn_input.edge_index)

            total_top1 += top_k_accuracy(preds, data.y, k=1)
            total_top3 += top_k_accuracy(preds, data.y, k=3)
            total_top5 += top_k_accuracy(preds, data.y, k=5)
            total_samples += 1

            top1_pred = torch.topk(preds, k=1, dim=1).indices.squeeze().tolist()
            if isinstance(top1_pred, int):  # handle single-sample batch
                top1_pred = [top1_pred]

            for i in range(len(data.y)):
                predictions.append([data.y[i].item(), top1_pred[i]])

    print(f"Top-1 Accuracy: {total_top1 / total_samples:.4f}")
    print(f"Top-3 Accuracy: {total_top3 / total_samples:.4f}")
    print(f"Top-5 Accuracy: {total_top5 / total_samples:.4f}")

    results_df = pd.DataFrame(predictions, columns=["Ground Truth", "Top-1 Prediction"])
    filename = f"test_results_{name.replace(' ', '_').replace('-', '_')}_coverage.csv"
    results_df.to_csv(filename, index=False)
    print(f"Saved predictions to {filename}")

# Run evaluation for each test range
for name, (start, end) in test_ranges.items():
    evaluate_test_set(name, start, end)



===== Evaluating Train Range 0-5 (Files 0 to 5) =====


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Top-1 Accuracy: 0.8109
Top-3 Accuracy: 0.9896
Top-5 Accuracy: 0.9986
Saved predictions to test_results_Train_Range_0_5_coverage.csv

===== Evaluating Train Range 5-10 (Files 5 to 10) =====


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Top-1 Accuracy: 0.6190
Top-3 Accuracy: 0.8705
Top-5 Accuracy: 0.9064
Saved predictions to test_results_Train_Range_5_10_coverage.csv

===== Evaluating Test Range 100-105 (Files 100 to 105) =====


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Top-1 Accuracy: 0.4001
Top-3 Accuracy: 0.7244
Top-5 Accuracy: 0.8312
Saved predictions to test_results_Test_Range_100_105_coverage.csv

===== Evaluating Test Range 200-205 (Files 200 to 205) =====


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Top-1 Accuracy: 0.3516
Top-3 Accuracy: 0.6650
Top-5 Accuracy: 0.7626
Saved predictions to test_results_Test_Range_200_205_coverage.csv


Testing

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import DataLoader
from tensorflow.keras.models import load_model
from collections import defaultdict, Counter

from dataloader import build_graph_from_file, prepare_input_for_gnn, NUM_GATEWAYS
from model import Stage2GNN

# === Paths ===
cells_path = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\cells_with_gateways.csv"
data_file = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Final folder real data\file_data_00_00_00.csv"
STAGE1_MODEL_PATH = 'stage_1_model.h5'
STAGE2_MODEL_PATH = 'stage2_loop_gnn_model_with_coverage.pth'

# === Load Data ===
cells_df = pd.read_csv(cells_path)
original_df = pd.read_csv(data_file)

stage1_model = load_model(STAGE1_MODEL_PATH)

gnn_model = Stage2GNN(
    input_dim=3 + NUM_GATEWAYS * 3,
    sat_feature_dim=111,
    neighbor_feature_dim=NUM_GATEWAYS,
    hidden_dim=256,
    output_dim=NUM_GATEWAYS
)
gnn_model.load_state_dict(torch.load(STAGE2_MODEL_PATH))
gnn_model.eval()

# === Prepare Graph ===
graph = build_graph_from_file(data_file)
data_loader = DataLoader([graph], batch_size=1)

# === Setup ===
unique_sats = original_df['feed_sat'].drop_duplicates().reset_index(drop=True)
feed_sat_to_id = {sat: idx for idx, sat in enumerate(unique_sats)}
sat_to_indices = defaultdict(list)
for row_idx, feed_sat in enumerate(original_df["feed_sat"]):
    sat_to_indices[feed_sat].append(row_idx)

satellite_to_gateway = {}
gateway_to_sats = defaultdict(list)
cell_to_gateways = {}  # index → [closest, second]
for idx, row in cells_df.iterrows():
    cell_to_gateways[idx] = [row["closest_gw_id"], row["second_closest_gw_id"]]

# === Model Inference ===
top1_preds = []
with torch.no_grad():
    for data in data_loader:
        input_features = data.x[:, :57].cpu().numpy()
        stage1_preds = stage1_model.predict(input_features, verbose=0)
        top3_indices = np.argsort(stage1_preds, axis=1)[:, -3:]

        binary_preds = np.zeros_like(stage1_preds)
        for i, idx in enumerate(top3_indices):
            binary_preds[i, idx] = 1

        gnn_input = prepare_input_for_gnn(data, torch.from_numpy(binary_preds).float())
        gnn_output = gnn_model(gnn_input.x, gnn_input.edge_index)
        top1_preds = torch.argmax(gnn_output, dim=1).cpu().numpy()

        for row_idx, gateway in enumerate(top1_preds):
            feed_sat = original_df.iloc[row_idx]["feed_sat"]
            satellite_to_gateway[feed_sat] = gateway
            gateway_to_sats[gateway].append(feed_sat)

# === Fair Cell Assignment (Greedy) ===
satellite_to_cells = defaultdict(list)
assigned_cells = set()
used_gateways = set()

for gw, sats in gateway_to_sats.items():
    nearby_cells = cells_df[
        (cells_df["closest_gw_id"] == gw) | (cells_df["second_closest_gw_id"] == gw)
    ].index.tolist()

    if not sats or not nearby_cells:
        continue

    used_gateways.add(gw)
    sats = list(set(sats))  # Remove dups
    n_sats = len(sats)
    cell_chunks = np.array_split(nearby_cells, n_sats)

    for sat, chunk in zip(sats, cell_chunks):
        for cell in chunk:
            if cell not in assigned_cells:
                satellite_to_cells[sat].append(cell)
                assigned_cells.add(cell)

# === Summary ===
total_unique_sats = len(unique_sats)
total_cells = len(cells_df)
all_gateways = set(range(NUM_GATEWAYS))

assigned_sat_ids = set(satellite_to_cells.keys())
unassigned_sat_ids = sorted(set(feed_sat_to_id.keys()) - assigned_sat_ids)
unused_gateways = sorted(all_gateways - used_gateways)
unassigned_cells = sorted(set(cells_df.index) - assigned_cells)

# === Summary ===
print("\n=== Summary ===")
print(f"Unique Satellites: {total_unique_sats}")
print(f"Satellites With Assigned Cells: {len(assigned_sat_ids)} ({len(assigned_sat_ids) / total_unique_sats * 100:.2f}%)")
print(f"Satellites With No Assigned Cells: {len(unassigned_sat_ids)} ({len(unassigned_sat_ids) / total_unique_sats * 100:.2f}%)")

print(f"Gateways With Unassigned Cells: {len(unused_gateways)} ({len(unused_gateways) / NUM_GATEWAYS * 100:.2f}%)")
print(f"Total Gateways: {NUM_GATEWAYS}")
print(f"Used Gateways: {len(used_gateways)} ({len(used_gateways) / NUM_GATEWAYS * 100:.2f}%)")
print(f"Unused Gateways: {unused_gateways}")

print(f"Assigned Cells: {len(assigned_cells)} / {total_cells} ({len(assigned_cells) / total_cells * 100:.2f}%)")
print(f"Unassigned Cells: {len(unassigned_cells)} ({len(unassigned_cells) / total_cells * 100:.2f}%)")

# === Export Results ===
# === Export Results ===
output_rows = []
all_sat_ids = sorted(feed_sat_to_id.keys())  # Include all satellites, not just assigned ones

for sat_feed_val in all_sat_ids:
    row_indices = sat_to_indices.get(sat_feed_val, [])
    predicted_gateways = [top1_preds[i] for i in row_indices if i < len(top1_preds)]
    most_common_gateway = Counter(predicted_gateways).most_common(1)[0][0] if predicted_gateways else None
    assigned = satellite_to_cells.get(sat_feed_val, [])

    # Convert np.int64 values to int
    assigned_clean = [int(cell) for cell in assigned]

    actual_gateway = original_df.loc[original_df["feed_sat"] == sat_feed_val, "gw"].iloc[0]
    actual_cell_ids = original_df.loc[original_df["feed_sat"] == sat_feed_val, "cell_id"].tolist()

    output_rows.append({
        "unique_satellite_id": sat_feed_val,
        "predicted_gateway": most_common_gateway,
        "actual_gateway_id": actual_gateway,
        "assigned_cells": assigned_clean,
        "actual_cell_ids": actual_cell_ids
    })

output_df = pd.DataFrame(output_rows)
output_df.to_csv("satellite_to_cells_mapping.csv", index=False)

# === Output preview ===
print("\n=== Output File Preview ===")
print(output_df.head())


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)



=== Summary ===
Unique Satellites: 112
Satellites With Assigned Cells: 84 (75.00%)
Satellites With No Assigned Cells: 28 (25.00%)
Gateways With Unassigned Cells: 0 (0.00%)
Total Gateways: 54
Used Gateways: 54 (100.00%)
Unused Gateways: []
Assigned Cells: 4569 / 4569 (100.00%)
Unassigned Cells: 0 (0.00%)

=== Output File Preview ===
   unique_satellite_id  predicted_gateway  actual_gateway_id  \
0                    5                  2                  2   
1                    7                 34                 34   
2                    8                 29                 29   
3                   28                  3                  2   
4                   29                 21                 21   

                                      assigned_cells  \
0                                                 []   
1                                                 []   
2  [92, 108, 163, 212, 303, 375, 440, 536, 598, 7...   
3              [144, 347, 483, 611, 848, 1081, 1139]   


Testing with cells 

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from collections import defaultdict, Counter
from tensorflow.keras.models import load_model
from torch_geometric.data import DataLoader
from dataloader import build_graph_from_file, prepare_input_for_gnn, NUM_GATEWAYS
from model2 import Stage2GNN  # Use model2 for coverage-aware model

# === Setup Paths ===
DATA_FOLDER = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Final folder real data"
CELLS_PATH = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\cells_with_gateways.csv"
GW_PATH = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\df_gw.csv"
STAGE1_MODEL_PATH = 'stage_1_model.h5'
STAGE2_MODEL_PATH = 'stage2_loop_gnn_model_with_coverage.pth'

# === Load Models ===
stage1_model = load_model(STAGE1_MODEL_PATH)
gnn_model = Stage2GNN(
    input_dim=3 + NUM_GATEWAYS * 3,
    sat_feature_dim=111,
    neighbor_feature_dim=NUM_GATEWAYS,
    hidden_dim=256,
    output_dim=NUM_GATEWAYS,
    dropout=0.3,
    use_residual=True
)
gnn_model.load_state_dict(torch.load(STAGE2_MODEL_PATH))
gnn_model.eval()

# === Prepare Output Directory ===
os.makedirs("results_with_cells_model2", exist_ok=True)

# === Select Files: 1st, 5th, 100th ===
file_list = sorted([f for f in os.listdir(DATA_FOLDER) if f.endswith('.csv')])
selected_indices = [0, 4, 99]
selected_files = [os.path.join(DATA_FOLDER, file_list[i]) for i in selected_indices if i < len(file_list)]

summary_stats = []
cells_df = pd.read_csv(CELLS_PATH)
gw_df = pd.read_csv(GW_PATH)

# === Colors for plotting ===
base_colors = list(mcolors.TABLEAU_COLORS.values())
np.random.seed(42)
np.random.shuffle(base_colors)
gateway_color_map = {i: base_colors[i % len(base_colors)] for i in range(NUM_GATEWAYS)}

def lighten_color(color, amount=0.5):
    import colorsys
    try:
        c = mcolors.cnames[color]
    except:
        c = color
    r, g, b = mcolors.to_rgb(c)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    r, g, b = colorsys.hls_to_rgb(h, min(1, l + amount * (1 - l)), s)
    return r, g, b

# === Main Loop for Each File ===
for file_path in selected_files:
    file_name = os.path.basename(file_path)
    original_df = pd.read_csv(file_path)
    graph = build_graph_from_file(file_path)
    data_loader = DataLoader([graph], batch_size=1)

    unique_sats = original_df['feed_sat'].drop_duplicates().reset_index(drop=True)
    feed_sat_to_id = {sat: idx for idx, sat in enumerate(unique_sats)}
    sat_to_indices = defaultdict(list)
    for row_idx, feed_sat in enumerate(original_df["feed_sat"]):
        sat_to_indices[feed_sat].append(row_idx)

    satellite_to_gateway = {}
    gateway_to_sats = defaultdict(list)
    cell_to_gateways = {idx: [row["closest_gw_id"], row["second_closest_gw_id"]] for idx, row in cells_df.iterrows()}

    # === Inference ===
    top1_preds = []
    with torch.no_grad():
        for data in data_loader:
            input_features = data.x[:, :57].cpu().numpy()
            stage1_preds = stage1_model.predict(input_features, verbose=0)
            top3_indices = np.argsort(stage1_preds, axis=1)[:, -3:]
            binary_preds = np.zeros_like(stage1_preds)
            for i, idx in enumerate(top3_indices):
                binary_preds[i, idx] = 1

            gnn_input = prepare_input_for_gnn(data, torch.from_numpy(binary_preds).float())
            gnn_output = gnn_model(gnn_input.x, gnn_input.edge_index)
            top1_preds = torch.argmax(gnn_output, dim=1).cpu().numpy()

            for row_idx, gateway in enumerate(top1_preds):
                feed_sat = original_df.iloc[row_idx]["feed_sat"]
                satellite_to_gateway[feed_sat] = gateway
                gateway_to_sats[gateway].append(feed_sat)

    # === Cell Assignment ===
    satellite_to_cells = defaultdict(list)
    assigned_cells = set()
    used_gateways = set()

    for gw, sats in gateway_to_sats.items():
        nearby_cells = cells_df[(cells_df["closest_gw_id"] == gw) | (cells_df["second_closest_gw_id"] == gw)].index.tolist()
        if not sats or not nearby_cells:
            continue
        used_gateways.add(gw)
        sats = list(set(sats))
        cell_chunks = np.array_split(nearby_cells, len(sats))
        for sat, chunk in zip(sats, cell_chunks):
            for cell in chunk:
                if cell not in assigned_cells:
                    satellite_to_cells[sat].append(cell)
                    assigned_cells.add(cell)

    # === Save Mapping ===
    output_rows = []
    all_sat_ids = sorted(feed_sat_to_id.keys())
    for sat_feed_val in all_sat_ids:
        row_indices = sat_to_indices.get(sat_feed_val, [])
        predicted_gateways = [top1_preds[i] for i in row_indices if i < len(top1_preds)]
        most_common_gateway = Counter(predicted_gateways).most_common(1)[0][0] if predicted_gateways else None
        assigned = satellite_to_cells.get(sat_feed_val, [])
        assigned_clean = [int(cell) for cell in assigned]
        actual_gateway = original_df.loc[original_df["feed_sat"] == sat_feed_val, "gw"].iloc[0]
        actual_cell_ids = original_df.loc[original_df["feed_sat"] == sat_feed_val, "cell_id"].tolist()
        output_rows.append({
            "unique_satellite_id": sat_feed_val,
            "predicted_gateway": most_common_gateway,
            "actual_gateway_id": actual_gateway,
            "assigned_cells": assigned_clean,
            "actual_cell_ids": actual_cell_ids
        })

    mapping_df = pd.DataFrame(output_rows)
    mapping_path = f"results_with_cells_model2/mapping_{file_name.replace('.csv', '')}.csv"
    mapping_df.to_csv(mapping_path, index=False)

    # === Summary Stats ===
    summary_stats.append({
        "File Name": file_name,
        "Rows": len(original_df),
        "Columns": original_df.shape[1],
        "Unique Satellites": original_df["feed_sat"].nunique(),
        "Unique Gateways": original_df["gw"].nunique(),
        "Unique Cells": original_df["cell_id"].nunique(),
        "Assigned Cells": len(assigned_cells),
        "Unassigned Cells": len(cells_df) - len(assigned_cells),
        "Used Gateways": len(used_gateways),
        "Unused Gateways": NUM_GATEWAYS - len(used_gateways)
    })

    # === Visualization ===
    def plot_map(title, save_name, include_sats=True, show_lines=False):
        plt.figure(figsize=(12, 10))
        for _, row in gw_df.iterrows():
            plt.scatter(row['longitude'], row['latitude'], marker='^',
                        color=gateway_color_map.get(row['gw_id'], 'gray'), edgecolor='black', s=150)

        for _, row in mapping_df.iterrows():
            pred_gw = row['predicted_gateway']
            color = lighten_color(gateway_color_map.get(pred_gw, 'gray'), amount=0.4)
            if include_sats:
                sat_row = original_df[original_df['feed_sat'] == row['unique_satellite_id']].drop_duplicates('feed_sat')
                if not sat_row.empty:
                    lng, lat = sat_row.iloc[0][['Longitude', 'Latitude']]
                    plt.scatter(lng, lat, marker='s', c=[color], edgecolor='black', s=80)

            for cell_idx in row['assigned_cells']:
                lng, lat = cells_df.loc[cell_idx, ['lng', 'lat']]
                plt.scatter(lng, lat, c=[color], s=10, alpha=0.7)

            if show_lines:
                pred_row = gw_df[gw_df['gw_id'] == row['predicted_gateway']]
                act_row = gw_df[gw_df['gw_id'] == row['actual_gateway_id']]
                if not pred_row.empty and not act_row.empty:
                    pred_lng, pred_lat = pred_row.iloc[0][['longitude', 'latitude']]
                    act_lng, act_lat = act_row.iloc[0][['longitude', 'latitude']]
                    plt.plot([pred_lng, act_lng], [pred_lat, act_lat], linestyle='--', color='gray')

        plt.title(title)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"results_with_cells_model2/{save_name}.png", dpi=300)
        plt.close()

    plot_map("Predicted Satellites and Gateways", f"map1_{file_name.replace('.csv', '')}")
    plot_map("Predicted Gateways and Assigned Cells", f"map2_{file_name.replace('.csv', '')}", include_sats=False)
    plot_map("Predicted vs Actual Gateways", f"map3_{file_name.replace('.csv', '')}", include_sats=False, show_lines=True)

# === Save All Summary ===
summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv("results_with_cells_model2/summary_stats.csv", index=False)
print("✅ Completed inference, visualization, and saved outputs for selected files (model2).")


c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\aruna\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


✅ Completed inference, visualization, and saved outputs for selected files (model2).
